# Usage Example

In this notebook we will show the most basic usage of **TGAN** in order to generate samples from a
given dataset.

### 1. Load the data

The first step is to load the data wich we will use to fit TGAN. In order to do so, we will first
import the function `tgan.data.load_data` and call it with the name the dataset that we want to load.

In this case, we will load the `census` dataset, which we will use during the subsequent steps, and obtain two objects:

1. `data` will contain a `pandas.DataFrame` with the table of data from the `census` dataset ready to be used to fit the model.

2. `continous_columns` will contain a `list` with the indices of continuous columns.

In [41]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [42]:
import pandas as pd
from tgan.model import TGANModel
import tensorflow as tf

In [26]:
d = pd.read_csv('../data/Ticket/Ticket_cat.csv', sep=';')

In [27]:
d.head()

,ItinID,MktID,MktCoupons,Year,Quarter,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,Origin,OriginCountry,...,OpCarrier,BulkFare,Passengers,MktFare,MktDistance,MktDistanceGroup,MktMilesFlown,NonStopMiles,ItinGeoType,MktGeoType
0,201513772623,20151377262301,1,2015,1,12892,1289203,32575,LAX,US,...,WN,0.0,12.0,120.0,1242.0,3,1242.0,1242.0,2,2
1,201513772624,20151377262401,1,2015,1,12892,1289203,32575,LAX,US,...,WN,0.0,1.0,121.0,1242.0,3,1242.0,1242.0,2,2
2,201513772625,20151377262501,1,2015,1,12892,1289203,32575,LAX,US,...,WN,0.0,1.0,126.0,1242.0,3,1242.0,1242.0,2,2
3,201513772626,20151377262601,1,2015,1,12892,1289203,32575,LAX,US,...,WN,0.0,25.0,129.0,1242.0,3,1242.0,1242.0,2,2
4,201513772627,20151377262701,1,2015,1,12892,1289203,32575,LAX,US,...,WN,0.0,3.0,131.0,1242.0,3,1242.0,1242.0,2,2


In [32]:
idxs = []
for col in d._get_numeric_data().columns:
    idxs.append(d.columns.get_loc(col))
print(idxs)

[0, 1, 2, 3, 4, 5, 6, 7, 10, 13, 14, 15, 16, 19, 22, 25, 27, 32, 33, 34, 35, 36, 37, 38, 39, 40]


In [37]:
def get_data(ds, drop=None):
    d = pd.read_csv(f'../data/{ds}/{ds}_cat.csv', sep=';')
    if drop is not None:
        d.drop(drop, axis=1)
        
    continuous_columns = []
    for col in d._get_numeric_data().columns:
        if len(d[col].unique()) > 20:
            continuous_columns.append(d.columns.get_loc(col))
    return d, continuous_columns

In [40]:
d, cont = get_data('Ticket')
d.head(), cont

(         ItinID           MktID  MktCoupons  Year  Quarter  OriginAirportID  \
 0  201513772623  20151377262301           1  2015        1            12892   
 1  201513772624  20151377262401           1  2015        1            12892   
 2  201513772625  20151377262501           1  2015        1            12892   
 3  201513772626  20151377262601           1  2015        1            12892   
 4  201513772627  20151377262701           1  2015        1            12892   
 
    OriginAirportSeqID  OriginCityMarketID Origin OriginCountry  ...  \
 0             1289203               32575    LAX            US  ...   
 1             1289203               32575    LAX            US  ...   
 2             1289203               32575    LAX            US  ...   
 3             1289203               32575    LAX            US  ...   
 4             1289203               32575    LAX            US  ...   
 
    OpCarrier BulkFare Passengers  MktFare  MktDistance  MktDistanceGroup  \
 0     

In [3]:
d = pd.read_csv('../data/berka/berka_cat.csv', sep=';')

/home/bauke/anaconda3/envs/p3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
d = d.drop(['trans_bank_partner', 'trans_account_partner'], axis=1)

In [5]:
d.head()

,trans_id,account_id,trans_amount,balance_after_trans,trans_type,trans_operation,trans_k_symbol,trans_date
0,695247,2378,700.0,700.0,CREDIT,CREDIT_IN_CASH,UNKNOWN,0
1,171812,576,900.0,900.0,CREDIT,CREDIT_IN_CASH,UNKNOWN,0
2,207264,704,1000.0,1000.0,CREDIT,CREDIT_IN_CASH,UNKNOWN,0
3,1117247,3818,600.0,600.0,CREDIT,CREDIT_IN_CASH,UNKNOWN,0
4,579373,1972,400.0,400.0,CREDIT,CREDIT_IN_CASH,UNKNOWN,1


In [44]:
continuous_columns = [0, 1, 2, 3, 7]

In [45]:
continuous_columns

[0, 1, 2, 3, 7]

### 2. Create a TGAN instance

The next step is to import TGAN and create an instance of the model.

To do so, we need to import the `tgan.model.TGANModel` class and call it.

This will create a TGAN instance with the default parameters.


In [8]:
# from comet_ml import Experiment
# experiment = Experiment(api_key="49HGMPyIKjokHwg2pVOKWTG67", project_name="gan-performance", workspace="baukebrenninkmeijer")

In [17]:
tf.keras.backend.clear_session()

In [46]:
tgan = TGANModel(continuous_columns, restore_session=False,  max_epoch=50, steps_per_epoch=1000, batch_size=1000)

ValueError: Comet.ml requires an API key. Please provide as the first argument to Experiment(api_key) or as an environment variable named COMET_API_KEY 

### 3. Fit the model

The third step is to pass the data that we have loaded previously to the `TGANModel.fit` method to
start the fitting.

This process will not return anything, however, the progress of the fitting will be printed into screen.

**NOTE** Depending on the performance of the system you are running, and the parameters selected
for the model, this step can take up to a few hours.


In [19]:
tgan.fit(d)
model_path = 'demo/my_model'

tgan.save(model_path)

[0522 08:30:37 @input_source.py:222] Setting up the queue 'QueueInput/input_queue' for CPU prefetching ...
(<tf.Tensor 'QueueInput/input_deque:0' shape=(1000, 1) dtype=float32>, <tf.Tensor 'QueueInput/input_deque:1' shape=(1000, 5) dtype=float32>, <tf.Tensor 'QueueInput/input_deque:2' shape=(1000, 1) dtype=float32>, <tf.Tensor 'QueueInput/input_deque:3' shape=(1000, 5) dtype=float32>, <tf.Tensor 'QueueInput/input_deque:4' shape=(1000, 1) dtype=float32>, <tf.Tensor 'QueueInput/input_deque:5' shape=(1000, 5) dtype=float32>, <tf.Tensor 'QueueInput/input_deque:6' shape=(1000, 1) dtype=float32>, <tf.Tensor 'QueueInput/input_deque:7' shape=(1000, 5) dtype=float32>, <tf.Tensor 'QueueInput/input_deque:8' shape=(1000, 1) dtype=int32>, <tf.Tensor 'QueueInput/input_deque:9' shape=(1000, 1) dtype=int32>, <tf.Tensor 'QueueInput/input_deque:10' shape=(1000, 1) dtype=int32>, <tf.Tensor 'QueueInput/input_deque:11' shape=(1000, 1) dtype=float32>, <tf.Tensor 'QueueInput/input_deque:12' shape=(1000, 5) d

Select Action: k (keep) / d (delete) / q (quit): k


[0522 08:32:14 @logger.py:83] Existing log file 'output/logs/log.log' backuped to 'output/logs/log.log.0522-083214'
[0522 08:32:14 @logger.py:90] Argv: /home/bauke/anaconda3/envs/p3.6/lib/python3.6/site-packages/ipykernel_launcher.py -f /run/user/1000/jupyter/kernel-5db58c25-eae8-49ea-97e0-1e3ec0fac56c.json
[0522 08:32:14 @model_utils.py:67] List of Trainable Variables: 
name                              shape           #elements
--------------------------------  ------------  -----------
gen/LSTM/go:0                     [1, 100]              100
gen/LSTM/lstm_cell/kernel:0       [500, 400]         200000
gen/LSTM/lstm_cell/bias:0         [400]                 400
gen/LSTM/00/FC/W:0                [100, 100]          10000
gen/LSTM/00/FC/b:0                [100]                 100
gen/LSTM/00/FC2/W:0               [100, 1]              100
gen/LSTM/00/FC2/b:0               [1]                     1
gen/LSTM/00/attw:0                [1, 1, 1]               1
gen/LSTM/01/FC/W:0        

100%|##########|1000/1000[04:24<00:00, 1.18it/s]

[0522 08:36:42 @base.py:285] Epoch 1 (global_step 1000) finished, time:4 minutes 24 seconds.


...

100%|##########|1000/1000[04:19<00:00, 3.85it/s]

[0522 08:41:03 @base.py:285] Epoch 2 (global_step 2000) finished, time:4 minutes 19 seconds.
[0522 08:41:03 @saver.py:79] Model saved to output/model/model-2000.
[0522 08:41:03 @monitor.py:467] GAN_loss/discrim/GP: 69424
[0522 08:41:03 @monitor.py:467] GAN_loss/discrim/d_loss: 6.9423e+05
[0522 08:41:03 @monitor.py:467] GAN_loss/gen/g_loss: 3.7221
[0522 08:41:03 @monitor.py:467] QueueInput/queue_size: 50
[0522 08:41:03 @base.py:275] Start Epoch 3 ...



 59%|#####8    |587/1000[02:32<01:47, 3.84it/s]

[0522 08:43:36 @base.py:293] Detected Ctrl-C and exiting main loop.


KeyboardInterrupt: 

### 4. Sample new data

After the model has been fit, we are ready to generate new samples by calling the `TGANModel.sample`
method passing it the desired amount of samples.

The returned object, `samples`, is a `pandas.DataFrame` containing a table of synthetic data with
the same format as the input data and 1000 rows as we requested.

In [14]:
num_samples = 100000

samples = tgan.sample(num_samples)

samples.head(20)

?|399/?[00:03<00:00,106.51it/s]


,trans_id,account_id,trans_amount,balance_after_trans,trans_type,trans_operation,trans_k_symbol,trans_date
0,8.313015e+05,2412.346404,-697.238617,62372.132949,WITHDRAWAL,WITHDRAWAL_IN_CASH,UNKNOWN,1226.659261
1,2.807175e+05,537.811549,25009.036967,62272.442144,CREDIT,CREDIT_IN_CASH,UNKNOWN,2011.850061
2,4.993796e+05,1384.407952,-666.895970,22699.081086,WITHDRAWAL,REMITTANCE_TO_OTHER_BANK,INSURANCE_PAYMENT,1937.274590
3,2.533511e+05,417.378888,775.283616,53160.347749,WITHDRAWAL,REMITTANCE_TO_OTHER_BANK,INSURANCE_PAYMENT,1095.971931
4,1.957608e+05,461.486269,-52.909450,38055.185932,WITHDRAWAL,WITHDRAWAL_IN_CASH,PAYMENT_FOR_STATEMENT,1920.859350
5,3.602992e+06,986.257363,7.028790,9005.636065,CREDIT,UNKNOWN,INTEREST_CREDITED,2171.315480
6,1.126110e+06,3481.758244,-1021.417024,43876.876451,WITHDRAWAL,REMITTANCE_TO_OTHER_BANK,INSURANCE_PAYMENT,1878.332241
7,3.632089e+06,2301.954204,2.555189,7973.871042,CREDIT,UNKNOWN,INTEREST_CREDITED,1992.822675
8,3.657328e+06,1695.505997,152.640724,25883.250439,CREDIT,UNKNOWN,INTEREST_CREDITED,1754.966151
9,4.223307e+05,1129.850558,726.509534,7518.877092,CREDIT,COLLECTION_FROM_OTHER_BANK,OLD_AGE_PENSION,1939.918932


In [15]:
p = samples.copy()
p[['trans_id', 'account_id', 'trans_amount', 'balance_after_trans', 'trans_date']] = p[['trans_id', 'account_id', 'trans_amount', 'balance_after_trans', 'trans_date']].astype('int')

In [16]:
p.to_csv('samples/berka_sample_wgan.csv', index=False)

In [33]:
len(samples)

1000

### 5. Save and Load a model

In the steps above we saw that the fitting process is slow, so we probably would like to avoid having to fit every we want to generate samples. Instead we can fit a model once, save it, and load it every time we want to sample new data.

If we have a fitted model, we can save it by calling the `TGANModel.save` method, that only takes
as argument the path to store the model into. Similarly, the `TGANModel.load` allows to load a model stored on disk by passing as argument a path where the model is stored.

At this point we could use this model instance to generate more samples.

In [8]:
model_path = 'model/berka_wgan_100x1000'

tgan.save(model_path)

[0520 11:02:32 @model.py:813] Model saved successfully.


In [19]:
model_path = 'model/berka_wgan_100x1000'
new_tgan = TGANModel.load(model_path)

(<tf.Tensor 'input00value:0' shape=(1000, 1) dtype=float32>, <tf.Tensor 'input00cluster:0' shape=(1000, 5) dtype=float32>, <tf.Tensor 'input01value:0' shape=(1000, 1) dtype=float32>, <tf.Tensor 'input01cluster:0' shape=(1000, 5) dtype=float32>, <tf.Tensor 'input02value:0' shape=(1000, 1) dtype=float32>, <tf.Tensor 'input02cluster:0' shape=(1000, 5) dtype=float32>, <tf.Tensor 'input03value:0' shape=(1000, 1) dtype=float32>, <tf.Tensor 'input03cluster:0' shape=(1000, 5) dtype=float32>, <tf.Tensor 'input04:0' shape=(1000, 1) dtype=int32>, <tf.Tensor 'input05:0' shape=(1000, 1) dtype=int32>, <tf.Tensor 'input06:0' shape=(1000, 1) dtype=int32>, <tf.Tensor 'input07value:0' shape=(1000, 1) dtype=float32>, <tf.Tensor 'input07cluster:0' shape=(1000, 5) dtype=float32>)
[0524 12:20:10 @collection.py:165] These collections were modified but restored in : (tf.GraphKeys.SUMMARIES: 0->7)
[0524 12:20:10 @sessinit.py:87] WRN The following variables are in the checkpoint, but not found in the graph: glo

In [20]:
num_samples = 100000
new_samples = new_tgan.sample(num_samples)

new_samples.head(3)

 10%|9         |99/1000[00:02<00:24,36.67it/s]


,trans_id,account_id,trans_amount,balance_after_trans,trans_type,trans_operation,trans_k_symbol,trans_date
0,624133.772715,2091.647880,7601.496306,11070.709384,WITHDRAWAL,WITHDRAWAL_IN_CASH,UNKNOWN,1760.831643
1,171180.893366,574.151060,12276.828773,61765.710178,WITHDRAWAL,WITHDRAWAL_IN_CASH,UNKNOWN,2076.040168
2,511430.606576,1715.231147,8919.582528,52761.409552,WITHDRAWAL,REMITTANCE_TO_OTHER_BANK,HOUSEHOLD,993.931269


In [21]:
p = new_samples.copy()
p[p._get_numeric_data().columns] = p[p._get_numeric_data().columns].astype('int')

In [22]:
p.to_csv('samples/berka_sample_wgan_100.csv', index=False)

In [16]:
len(new_samples)

10000

## Loading custom datasets

In the previous steps we used some demonstration data but we did not show how to load your own dataset.

In order to do so you can use `pandas.read_csv` by passing it the path to the CSV file that you want to load.

Additionally, you will need to create 0-indexed list of columns indices to be considered continuous.

For example, if we want to load a local CSV file, `path/to/my.csv`, that has as continuous columns their first 4 columns, that is, indices `[0,1,2,3]`, we would do it like this:

In [ ]:
import pandas as pd

data = pd.read_csv('data/census.csv')

continuous_columns = [0,1,2,3]

## Model Parameters

If you want to change the default behavior of TGANModel, such as as different `batch_size` or
`num_epochs`, you can do so by passing different arguments when creating the instance. Have b

### Model general behavior

* continous_columns (`list[int]`, required): List of columns to be considered continuous.
* output (`str`, default=`output`): Path to store the model and its artifacts.
* gpu (`list[str]`, default=`[]`): Comma separated list of GPU(s) to use.

### Neural network definition and fitting

* max_epoch (`int`, default=`100`): Number of epochs to use during training.
* steps_per_epoch (`int`, default=`10000`): Number of steps to run on each epoch.
* save_checkpoints(`bool`, default=True): Whether or not to store checkpoints of the model after each training epoch.
* restore_session(`bool`, default=True): Whether or not continue training from the last checkpoint.
* batch_size (`int`, default=`200`): Size of the batch to feed the model at each step.
* z_dim (`int`, default=`100`): Number of dimensions in the noise input for the generator.
* noise (`float`, default=`0.2`): Upper bound to the gaussian noise added to categorical columns.
* l2norm (`float`, default=`0.00001`): L2 reguralization coefficient when computing losses.
* learning_rate (`float`, default=`0.001`): Learning rate for the optimizer.
* num_gen_rnn (`int`, default=`400`):
* num_gen_feature (`int`, default=`100`): Number of features of in the generator.
* num_dis_layers (`int`, default=`2`):
* num_dis_hidden (`int`, default=`200`):
* optimizer (`str`, default=`AdamOptimizer`): Name of the optimizer to use during `fit`, possible
  values are: [`GradientDescentOptimizer`, `AdamOptimizer`, `AdadeltaOptimizer`].

If we wanted to create an identical instance to the one created on step 2, but passing the arguments in a explicit way we will do something like this:

In [ ]:
tgan = TGANModel(
    continuous_columns,
    output='output',
    gpu=None,
    max_epoch=5,
    steps_per_epoch=10000,
    save_checkpoints=True,
    restore_session=True,
    batch_size=200,
    z_dim=200,
    noise=0.2,
    l2norm=0.00001,
    learning_rate=0.001,
    num_gen_rnn=100,
    num_gen_feature=100,
    num_dis_layers=1,
    num_dis_hidden=100,
    optimizer='AdamOptimizer'
)

## Command-line interface

We include a command-line interface that allows users to access TGAN functionality. Currently only one action is supported.

### Random hyperparameter search

#### Input

To run random searchs for the best model hyperparameters for a given dataset, we will need:

* A dataset, in a csv file, without any missing value, only columns of type `bool`, `str`, `int` or
  `float` and only one type for column, as specified in [Data Format Input](#data-format-input).

* A JSON file containing the configuration for the search. This configuration shall contain:

  * `name`: Name of the experiment. A folder with this name will be created.
  * `num_random_search`: Number of iterations in hyper parameter search.
  * `train_csv`: Path to the csv file containing the dataset.
  * `continuous_cols`: List of column indices, starting at 0, to be considered continuous.
  * `epoch`: Number of epoches to train the model.
  * `steps_per_epoch`: Number of optimization steps in each epoch.
  * `sample_rows`: Number of rows to sample when evaluating the model.

You can see an example of such a json file in [examples/config.json](examples/config.json), which you
can download and use as a template.

#### Execution

Once we have prepared everything we can launch the random hyperparameter search with this command:

``` bash
tgan experiments config.json results.json
```

Where the first argument, `config.json`,  is the path to your configuration JSON, and the second,
`results.json`, is the path to store the summary of the execution.

This will run the random search, wich basically consist of the folling steps:

1. We fetch and split our data between test and train.
2. We randomly select the hyperparameters to test.
3. Then, for each hyperparameter combination, we train a TGAN model using the real training data T
   and generate a synthetic training dataset Tsynth.
4. We then train machine learning models on both the real and synthetic datasets.
5. We use these trained models on real test data and see how well they perform.

#### Output

One the experiment has finished, the following can be found:

* A JSON file, in the example above called `results.json`, containing a summary of the experiments.
  This JSON will contain a key for each experiment `name`, and on it, an array of length
  `num_random_search`, with the selected parameters and its evaluation score. For a configuration
  like the example, the summary will look like this:

``` python
{
    'census': [
        {
            "steps_per_epoch" : 10000,
            "num_gen_feature" : 300,
            "num_dis_hidden" : 300,
            "batch_size" : 100,
            "num_gen_rnn" : 400,
            "score" : 0.937802280415988,
            "max_epoch" : 5,
            "num_dis_layers" : 4,
            "learning_rate" : 0.0002,
            "z_dim" : 100,
            "noise" : 0.2
        },
        ... # 9 more nodes
    ]
}
```

* A set of folders, each one names after the `name` specified in the JSON configuration, contained
in the `experiments` folder. In each folder, sampled data and the models can be found. For a configuration
like the example, this will look like this:

```
experiments/
  census/
    data/       # Sampled data with each of the models in the random search.
    model_0/
      logs/     # Training logs
      model/    # Tensorflow model checkpoints
    model_1/    # 9 more folders, one for each model in the random search
    ...
```

## Citation

If you use TGAN, please cite the following work:

> Lei Xu, Kalyan Veeramachaneni. 2018. Synthesizing Tabular Data using Generative Adversarial Networks.

```LaTeX
@article{xu2018synthesizing,
  title={Synthesizing Tabular Data using Generative Adversarial Networks},
  author={Xu, Lei and Veeramachaneni, Kalyan},
  journal={arXiv preprint arXiv:1811.11264},
  year={2018}
}
```
You can find the original paper [here](https://arxiv.org/pdf/1811.11264.pdf)